### IMPORTS

In [1]:
import json
import base64
import hashlib
import pkg_resources.py2_warn

In [2]:
from web3 import Web3

In [3]:
print('[X] Booting up...')

[X] Booting up...


### LOAD DATA FROM EXTERNAL FILES

In [4]:
with open('settings.json') as json_file:
    settings = json.load(json_file)

In [5]:
with open('identifier.json') as json_file:
    device = json.load(json_file)

In [6]:
with open('latest.json') as json_file:
    latest = json.load(json_file)

### IPFS & BLOCKCHAIN GATEWAY CONNECTIONS

In [7]:
class gateway:
    def __init__(self, foo):
        self.host = foo['host']
        self.port = foo['port']

In [8]:
blockchain = gateway(settings['gateways']['blockchain'])

In [9]:
ipfs = gateway(settings['gateways']['ipfs'])

In [10]:
print('[X] Gateways set!')

[X] Gateways set!


### BLOCKCHAIN PUBLIC/PRIVATE KEYS

In [11]:
class keypair:
    def __init__(self, foo):
        self.public = foo['public']
        self.private = foo['private']

In [12]:
keys = keypair(settings['keys'])

In [13]:
print('[X] Keys set!')

[X] Keys set!


### HASH DEVICE PARAMS FOR IDENTIFIER

In [14]:
def hash_id(data):
    
    # CONVERT TO BYTES
    to_bytes = str.encode(str(data))
    
    # HASH ENCODED DATA
    hashed = hashlib.sha224(to_bytes).hexdigest()
    
    return hashed

In [15]:
device_id = hash_id(device)

In [16]:
print('[X] Device identifier hashed!')

[X] Device identifier hashed!


### CONNECT TO CHAIN VIA WEBSOCKET

In [17]:
web3 = Web3(Web3.WebsocketProvider('ws://' + blockchain.host + ':' + blockchain.port))

In [18]:
if web3.isConnected():
    print('[X] Connected with gateway!')
else:
    print('[ ] Could not connect to gateway!')
    exit

[X] Connected with gateway!


### CONSTRUCT MANAGER CONTRACT

In [19]:
class manager:
    
    # ON LOAD..
    def __init__(self, block):
        
        # CONSTRUCT USABLE CONTRACT
        self.contract = web3.eth.contract(
            address = block['address'],
            abi = block['abi']
        )
        
        # SET ADDRESS REFERENCE
        self.address = block['address']
    
    # READ FROM CONTRACT
    def read(self, details):
        
        # WITH PARAMS
        if ('params' in details):
            return self.contract.functions[details['func']](details['params']).call()
        
        # WITHOUT PARAMS
        else:
            return self.contract.functions[details]().call()
    
    # WRITE TO CONTRACT
    def write(self, details):
        try:
            
            # CREATE BASE TRANSACTION
            tx = {
                'from': keys.public,
                'to': self.contract.address,
                'data': self.contract.encodeABI(
                    fn_name = details['func'],
                    args = details['params']
                )
            }
            
            # ESTIMATE GAS VALUE & STITCH IN REMAINING PROPS
            tx['gas'] = web3.eth.estimateGas(tx)
            tx['gasPrice'] = web3.toWei(20, 'gwei')
            tx['nonce'] = web3.eth.getTransactionCount(keys.public)

            # SIGN TRANSCTION WITH PRIVATE KEY
            signed = web3.eth.account.sign_transaction(tx,
                private_key = keys.private
            )

            # SEND THE TRANSACTION
            web3.eth.sendRawTransaction(signed.rawTransaction)

            # SUCCESS
            return True
        
        # IF THE TRANSACTION IS REVERTED, SHOW ERROR
        except ValueError as error:
            return error

### CONTRACT REFERENCES

In [20]:
user_manager = manager(latest['usermanager'])

In [21]:
device_manager = manager(latest['devicemanager'])

In [22]:
task_manager = manager(latest['taskmanager'])

In [23]:
token_manager = manager(latest['tokenmanager'])

In [24]:
print('[X] Manager contracts set!')

[X] Manager contracts set!


### INIT MANAGER CONTRACTS

In [25]:
def init_managers():
    
    # FLEXIBLE PARAMS
    token_price = 5000
    task_fee = 5
    
    # TOKEN MANAGER
    token_manager.write({
        'func': 'init',
        'params': [
            token_price,
            task_manager.address
        ]
    })
    
    # TASK MANAGER
    task_manager.write({
        'func': 'init',
        'params': [
            task_fee,
            user_manager.address,
            device_manager.address,
            token_manager.address
        ]
    })
    
    # USER MANAGER
    user_manager.write({
        'func': 'init',
        'params': [task_manager.address]
    })
    
    # DEVICE MANAGER
    device_manager.write({
        'func': 'init',
        'params': [
            user_manager.address,
            task_manager.address
        ]
    })
    
    return True

### PERFORM INITIALIZATION -- IF NECESSARY

In [26]:
# CHECK STATUS
initialized = token_manager.read('initialized')

In [27]:
initialized

False

In [28]:
# INIT HAS BEEN PERFORMED
if (initialized):
    print('[X] Manager contracts have already been initialized!')
    
# INIT NOW
else:
    result = init_managers()
    
    if (result):
        print('[X] Initializing manager contracts!')

[X] Initializing manager contracts!


### REGISTER A USER -- IF NECESSARY

In [29]:
def register_user():
    
    # REGISTER ETH WALLET
    user_manager.write({
        'func': 'add',
        'params': []
    })
    
    return True

In [30]:
user_check = user_manager.read({
    'func': 'fetch',
    'params': keys.public
})

In [31]:
# USER NOT FOUND
if (user_check == '0x0000000000000000000000000000000000000000'):
    
    # REGISTER NOW
    result = register_user()
    
    if (result):
        print('[X] Registered your wallet!')
    
# USER FOUND
else:
    print('[X] Your wallet is already registered!')

[X] Registered your wallet!


### REGISTER A DEVICE -- IF NECESSARY

In [32]:
def register_device(name):
    
    # REGISTER ETH WALLET
    device_manager.write({
        'func': 'add',
        'params': [device_id, name]
    })
    
    return True

In [33]:
device_check = device_manager.read({
    'func': 'fetch_device',
    'params': device_id
})

In [34]:
# USER NOT FOUND
if (device_check == '0x0000000000000000000000000000000000000000'):
    
    # REGISTER NOW
    result = register_device('foo')
    
    if (result):
        print('[X] Registered this device!')
    
# USER FOUND
else:
    print('[X] This device is already registered!')

[X] Registered this device!


### LOCATE DEVICE CONTRACT

In [35]:
device_contract = device_manager.read({
    'func': 'fetch_device',
    'params': device_id
})

In [36]:
device_contract

'0xf430f1F079A06be325b253Dd1Eb6e4695FBe88a8'